<a href="https://colab.research.google.com/github/AluminumShark/2024_Deep_Learning_with_Pytorch/blob/main/02_Convnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import numpy as np

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'DEVICE: {device}')

EPOCHS = 64
BATCH_SIZE = 128
LEARNING_RATE = 0.01

print(f'EPOCHS: {EPOCHS}')
print(f'BATCH_SIZE: {BATCH_SIZE}')
print(f'LEARNING_RATE: {LEARNING_RATE}')

DEVICE: cpu
EPOCHS: 64
BATCH_SIZE: 128
LEARNING_RATE: 0.01


In [3]:
# Load training into a single batch to compute mean and std

trainset = CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())

trainloader = DataLoader(trainset, batch_size=len(trainset), shuffle=False)

data = next(iter(trainloader))

mean = data[0].mean()
std = data[0].std()

transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

trainset = CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = CIFAR10(root='./data', train=False, download=True, transform=transform)

100%|██████████| 170M/170M [00:03<00:00, 52.5MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Files already downloaded and verified


In [4]:
model = nn.Sequential(
    nn.Conv2d(3, 64, 5, stride=2, padding=2),
    nn.BatchNorm2d(64),
    nn.ReLU(),

    nn.Conv2d(64, 64, 3, stride=2, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),

    nn.Flatten(),
    nn.Dropout(0.3),
    nn.Linear(64 * 8 * 8, 10)
)

conv_layer0 = model[0]
conv_layer1 = model[3]
output_layer = model[8]

# Initialize weights and biases
nn.init.kaiming_normal_(conv_layer0.weight)
nn.init.constant_(conv_layer0.bias, 0.0)
nn.init.kaiming_normal_(conv_layer1.weight)
nn.init.constant_(conv_layer1.bias, 0.0)
nn.init.xavier_uniform_(output_layer.weight)
nn.init.constant_(output_layer.bias, 0.0)


optimizer = torch.optim.Adam(model.parameters(), weight_decay=1e-4)
loss_fn = nn.CrossEntropyLoss()

In [5]:
# 移動模型到指定設備（CPU或GPU）
model = model.to(device)

# 訓練函式
def train(model, train_loader, optimizer, loss_fn, epoch):
    model.train()  # 設置模型為訓練模式
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, targets in train_loader:
        # 移動資料到指定設備
        inputs, targets = inputs.to(device), targets.to(device)

        # 清空梯度
        optimizer.zero_grad()

        # 前向傳播
        outputs = model(inputs)

        # 計算損失
        loss = loss_fn(outputs, targets)

        # 反向傳播和優化
        loss.backward()
        optimizer.step()

        # 累積損失和準確度
        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    epoch_loss = running_loss / total
    accuracy = 100. * correct / total
    print(f'Epoch {epoch+1}/{EPOCHS}, Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%')

# 測試函式
def evaluate(model, test_loader, loss_fn):
    model.eval()  # 設置模型為評估模式
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # 禁用梯度計算，加速和節省記憶體
        for inputs, targets in test_loader:
            # 移動資料到指定設備
            inputs, targets = inputs.to(device), targets.to(device)

            # 前向傳播
            outputs = model(inputs)

            # 計算損失
            loss = loss_fn(outputs, targets)
            running_loss += loss.item() * inputs.size(0)

            # 計算準確度
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    epoch_loss = running_loss / total
    accuracy = 100. * correct / total
    print(f'Test Loss: {epoch_loss:.4f}, Test Accuracy: {accuracy:.2f}%')

# 訓練和評估模型
for epoch in range(EPOCHS):
    train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False)

    # 執行訓練
    train(model, train_loader, optimizer, loss_fn, epoch)

    # 執行測試
    evaluate(model, test_loader, loss_fn)


Epoch 1/64, Loss: 1.7707, Accuracy: 36.85%
Test Loss: 1.4734, Test Accuracy: 45.87%


KeyboardInterrupt: 